# SETUP

## Import Libraries

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Read MongoDB

In [2]:
client = MongoClient('mongodb://localhost:27017')
db = client["test"]

## Initialize Output DataFrame

In [3]:
output_df = pd.DataFrame()

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [4]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [ ]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [6]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [7]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

In [8]:
team_list = [team for team in team_list if not np.isnan(team)]

### Complete Range of All Matches

In [9]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [10]:
output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

output_df["Auto Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Auto Speaker Notes Total Avg"] = sum([output_df.at[team, "Auto Speaker Notes Near Avg"], output_df.at[team, "Auto Speaker Notes Mid Avg"], output_df.at[team, "Auto Speaker Notes Far Avg"]])

In [11]:

grouped = matchapps_df.groupby("metadata.robotTeam")
output_df["Auto Speaker Notes Near Max"] = grouped["autoNotes.near"].max()
output_df["Auto Speaker Notes Mid Max"] = grouped["autoNotes.mid"].max()
output_df["Auto Speaker Notes Far Max"] = grouped["autoNotes.far"].max()
output_df["Auto Speaker Notes Total Max"] = (matchapps_df[["autoNotes.far", "autoNotes.mid", "autoNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max
metadata.robotTeam,,,,,,,,
368.0,0.333333,2.000000,0.555556,2.888889,1,4,2,6
597.0,0.625000,0.875000,0.000000,1.5,1,7,0,7
599.0,0.000000,0.000000,0.000000,0.0,0,0,0,0
687.0,2.428571,0.714286,0.142857,3.285714,4,3,1,4
702.0,0.428571,0.714286,0.000000,1.142857,1,3,0,3
968.0,0.000000,0.000000,0.000000,0.0,0,0,0,0
980.0,0.571429,0.428571,0.000000,1.0,1,1,0,2
1148.0,0.142857,0.142857,0.000000,0.285714,1,1,0,2
1159.0,0.250000,0.375000,0.000000,0.625,1,1,0,1


#### Auto Amp Notes Average

In [12]:
output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [13]:
output_df["Auto Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Auto Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Auto Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].max()

/tmp/ipykernel_41836/457572201.py:12: RuntimeWarning: invalid value encountered in scalar divide
  output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
/tmp/ipykernel_41836/457572201.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes


#### Auto All Notes Average

In [14]:
output_df["Auto Notes Avg"] = (output_df["Auto Notes Amp Avg"] +
                               output_df["Auto Speaker Notes Near Avg"] +
                               output_df["Auto Speaker Notes Mid Avg"] +
                               output_df["Auto Speaker Notes Far Avg"]) / 4

#### Average Auto points

In [15]:
output_df['Auto Points Avg'] = (5 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 2 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [16]:
output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

output_df["Tele Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Tele Speaker Notes Total Avg"] = sum([output_df.at[team, "Tele Speaker Notes Near Avg"], output_df.at[team, "Tele Speaker Notes Mid Avg"], output_df.at[team, "Tele Speaker Notes Far Avg"]])

In [17]:
output_df["Tele Speaker Notes Near Max"] = grouped["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = grouped["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = grouped["teleNotes.far"].max()
output_df["Tele Speaker Notes Total Max"] = (matchapps_df[["teleNotes.far", "teleNotes.mid", "teleNotes.near"]].sum(axis=1)).groupby(matchapps_df['metadata.robotTeam']).max()

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Auto Notes Avg,Auto Points Avg,Tele Speaker Notes Near Avg,Tele Speaker Notes Mid Avg,Tele Speaker Notes Far Avg,Tele Speaker Notes Total Avg,Tele Speaker Notes Near Max,Tele Speaker Notes Mid Max,Tele Speaker Notes Far Max,Tele Speaker Notes Total Max
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
368.0,0.333333,2.000000,0.555556,2.888889,1,4,2,6,0.000000,1.0,...,0.722222,26.444444,0.222222,2.888889,0.444444,3.555556,2,5,2,7
597.0,0.625000,0.875000,0.000000,1.5,1,7,0,7,0.000000,0.4,...,0.375000,13.375000,0.125000,1.875000,0.125000,2.125,1,6,1,7
599.0,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000000,inf,...,0.000000,1.875000,0.250000,0.125000,0.000000,0.375,2,1,0,2
687.0,2.428571,0.714286,0.142857,3.285714,4,3,1,4,0.000000,0.058824,...,0.821429,42.000000,6.428571,1.571429,0.000000,8.0,10,6,0,10
702.0,0.428571,0.714286,0.000000,1.142857,1,3,0,3,0.000000,3.333333,...,0.285714,22.285714,2.000000,1.714286,0.000000,3.714286,7,4,0,9
968.0,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0,0,0,0
980.0,0.571429,0.428571,0.000000,1.0,1,1,0,2,0.000000,0.0,...,0.250000,17.142857,2.857143,0.571429,0.000000,3.428571,7,2,0,7
1148.0,0.142857,0.142857,0.000000,0.285714,1,1,0,2,0.000000,0.0,...,0.071429,13.571429,0.285714,0.285714,0.142857,0.714286,1,1,1,2
1159.0,0.250000,0.375000,0.000000,0.625,1,1,0,1,0.000000,0.5,...,0.156250,8.875000,0.875000,0.750000,0.000000,1.625,3,3,0,3


#### Teleop Amp Notes Average

In [18]:
output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

 # CHANGE FORMAT LATER
output_df["Tele Max Amp Notes"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].max()

#### Teleop All Notes Average

In [19]:
output_df["Tele Notes Avg"] = (output_df["Tele Notes Amp Avg"] +
                               output_df["Tele Speaker Notes Near Avg"] +
                               output_df["Tele Speaker Notes Mid Avg"] +
                               output_df["Tele Speaker Notes Far Avg"]) / 4

 # CHANGE FORMAT LATER
output_df["Tele Max Notes"] = None

#### Teleop Percent of Missed Notes

In [20]:
output_df["Tele Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Tele Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Tele Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].max()

/tmp/ipykernel_41836/471659397.py:12: RuntimeWarning: invalid value encountered in scalar divide
  output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
/tmp/ipykernel_41836/471659397.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes


In [21]:
output_df["Tele Speaker Notes Near Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].max()
output_df["Tele Speaker Notes Mid Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].max()
output_df["Tele Speaker Notes Far Max"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].max()

#### Average Teleop Points

In [22]:
output_df['Tele Points Avg'] = (2 * (matchapps_df['teleNotes.near'] + matchapps_df['teleNotes.mid'] + matchapps_df['teleNotes.far']) + 1 * matchapps_df['teleNotes.amp']).groupby(matchapps_df['metadata.robotTeam']).mean()

In [23]:
# output_df['Endgame Points Avg'] =
#(np.where(matchapps_df['trapNotes']), 0, 1)
np.where(matchapps_df['climb'].isin(['source', 'center', 'amp']), 0, 3)


array([0, 0, 3, 0, 3, 3, 3, 0, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 3, 3, 3, 0, 3, 0, 3, 0,
       3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 0, 3, 0, 3,
       3, 3, 3, 0, 0, 0, 0, 3, 0, 3, 3, 3, 0, 3, 0, 3, 3, 3, 0, 0, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 0, 3,
       0, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 3, 3,
       3, 0, 3, 0, 0, 0, 3, 0, 0, 0, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0,
       3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 3, 0, 0, 3, 0,
       3, 0, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 3, 0,
       3, 3, 0, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3,
       3, 3, 0, 3, 0, 3, 0, 0, 3, 3, 3, 0, 3, 3, 3, 0, 0, 3, 0, 0, 3, 3,
       0, 3, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 0, 0, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 0, 0, 0, 3, 3, 3, 3, 3,

### Other Data Analysis Calculations

#### Trap Notes

In [24]:
output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [25]:
output_df["Percent of Successful Climbs"] = None
output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

#### Percent of Harmonized Climbs out of Successful Climbs

In [26]:
# output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

# output_df.index

# for team in team_list:
#     output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = output_df.at[team, "Harmonized Climbs Total"] / output_df.at[team, "Successful Climbs Total"]

#### Percent of Times Robot Left Starting Zone During Autos

In [27]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [28]:
# output_df["Endgame Custom Metric"] = ((output_df["Percent of Successful Climbs"] * 5) ** 2 + (output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (output_df["Trap Notes Avg"] * 5) ** 2) / 3
# output_df["Endgame Custom Metric"] = output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [29]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [ ]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

## Superapps Data Analysis

### Average Fouls per Match for Different Foul Types

In [31]:
output_df["Average In Robot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.insideRobot"].mean()
output_df["Average Protected Zone Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.protectedZone"].mean()
output_df["Average Pinning Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinning"].mean()
output_df["Average Multiple Pieces Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.multiplePieces"].mean()
output_df["Average Other Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.other"].mean()


### Average Total Fouls per Match

In [32]:
output_df["Average Fouls per Match"] = superapps_df[[
    'fouls.insideRobot', 'fouls.protectedZone', 'fouls.pinning', 'fouls.multiplePieces', 'fouls.other'
]].sum(axis = 1).groupby(superapps_df['metadata.robotTeam']).mean()
output_df['Average Fouls per Match']

metadata.robotTeam
368.0     0.111111
597.0     0.000000
599.0     0.714286
687.0     0.428571
702.0     0.000000
968.0     0.142857
980.0     0.000000
1148.0    0.166667
1159.0    0.000000
1160.0    0.125000
1197.0    0.000000
1452.0    0.571429
1515.0    0.500000
1661.0    0.875000
1759.0    0.000000
1836.0    0.333333
2443.0    0.000000
2465.0    0.285714
2584.0    0.125000
2637.0    0.428571
2710.0    0.250000
3408.0    0.142857
3473.0    0.375000
3759.0    0.000000
3859.0    0.166667
4079.0    0.142857
4123.0    0.142857
4201.0    0.111111
4501.0    1.500000
4999.0    0.250000
5089.0    0.250000
5199.0    0.250000
5669.0    0.142857
5857.0    0.571429
6000.0    0.571429
6658.0    0.142857
6904.0    0.250000
7042.0    0.000000
7137.0    0.375000
7230.0    0.000000
7415.0    0.142857
8898.0    0.571429
9408.0    0.000000
9564.0    1.375000
Name: Average Fouls per Match, dtype: float64

### Percent of Different Defense Types in Matches AND Main Defense Type

In [33]:
def safeDivide(a, b):
    return 0 if b == 0 else a / b

In [34]:
output_df["Percent of Matches with No Defense"] = None
output_df["Percent of Matches with Some Defense"] = None
output_df["Percent of Matches with Full Defense"] = None
output_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = superapps_df.loc[superapps_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    output_df.at[team, "Percent of Matches with No Defense"] = safeDivide(no_defense_count, defense_total_count)
    output_df.at[team, "Percent of Matches with Some Defense"] = safeDivide(some_defense_count, defense_total_count)
    output_df.at[team, "Percent of Matches with Full Defense"] = safeDivide(full_defense_count, defense_total_count)
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    output_df.at[team, "Main Defense Type"] = main_defense_type

## Comment Counts

In [35]:
exploded = superapps_df.explode('comments')

In [36]:
# Individual Counts
comment_count_columns = exploded.groupby('metadata.robotTeam')['comments'].value_counts().unstack(fill_value = 0)
for col in comment_count_columns:
    output_df[f'Total {col} Comments'] = comment_count_columns[col]

In [37]:
for comment in ['avoids_under_stage',
 'clogging',
 'effective_defense',
 'good_driving',
 'ineffective_defense',
 'okay_defense',
 'source_only',
 'sturdy_build',
 'weak_build']:
    output_df[f'Average {comment} comments'] = None

    for team in team_list:
        
        commentInMatches = superapps_df.loc[superapps_df["metadata.robotTeam"] == team, 'comments'].apply(lambda x: comment in x)

        output_df.at[team, f'Average {comment} comments'] = safeDivide(commentInMatches[commentInMatches].size, commentInMatches.size)

output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Total weak_build Comments,Average avoids_under_stage comments,Average clogging comments,Average effective_defense comments,Average good_driving comments,Average ineffective_defense comments,Average okay_defense comments,Average source_only comments,Average sturdy_build comments,Average weak_build comments
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
368.0,0.333333,2.000000,0.555556,2.888889,1,4,2,6,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.0
597.0,0.625000,0.875000,0.000000,1.5,1,7,0,7,0.000000,0.4,...,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
599.0,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000000,inf,...,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857,0.0,0.0,0.0
687.0,2.428571,0.714286,0.142857,3.285714,4,3,1,4,0.000000,0.058824,...,0.0,0.0,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0
702.0,0.428571,0.714286,0.000000,1.142857,1,3,0,3,0.000000,3.333333,...,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0
968.0,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000000,NaN,...,0.0,0.0,0.142857,0.142857,0.142857,0.0,0.0,0.0,0.0,0.0
980.0,0.571429,0.428571,0.000000,1.0,1,1,0,2,0.000000,0.0,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1148.0,0.142857,0.142857,0.000000,0.285714,1,1,0,2,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1159.0,0.250000,0.375000,0.000000,0.625,1,1,0,1,0.000000,0.5,...,0.0,0.0,0.142857,0.0,0.142857,0.0,0.0,0.0,0.0,0.0


In [38]:
# Single Object

comment_counts = exploded.groupby('metadata.robotTeam')['comments'].apply(lambda x: [x.value_counts().to_dict()]).apply(lambda x: x[0])
# Yes that is jank but it wouldn't dictify nicely otherwise

output_df['Comments'] = comment_counts


## Additionals

# FINALIZE AND SEND DATA

## Send Data

In [39]:
output_df['teamNumber'] = output_df.index

In [40]:
output_df.to_json("../server/static/output_analysis.json", orient='records', default_handler=str)
output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Speaker Notes Near Max,Auto Speaker Notes Mid Max,Auto Speaker Notes Far Max,Auto Speaker Notes Total Max,Auto Notes Amp Avg,Auto Percent of Missed Notes,...,Average clogging comments,Average effective_defense comments,Average good_driving comments,Average ineffective_defense comments,Average okay_defense comments,Average source_only comments,Average sturdy_build comments,Average weak_build comments,Comments,teamNumber
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
368.0,0.333333,2.000000,0.555556,2.888889,1,4,2,6,0.000000,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.222222,0.0,"{'great_driving': 3, 'sturdy_build': 2}",368.0
597.0,0.625000,0.875000,0.000000,1.5,1,7,0,7,0.000000,0.4,...,0.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,{'clogging': 1},597.0
599.0,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000000,inf,...,0.142857,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,"{'okay_defense': 1, 'clogging': 1}",599.0
687.0,2.428571,0.714286,0.142857,3.285714,4,3,1,4,0.000000,0.058824,...,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0,{'good_driving': 2},687.0
702.0,0.428571,0.714286,0.000000,1.142857,1,3,0,3,0.000000,3.333333,...,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,{'good_driving': 1},702.0
968.0,0.000000,0.000000,0.000000,0.0,0,0,0,0,0.000000,NaN,...,0.142857,0.142857,0.142857,0.0,0.0,0.0,0.0,0.0,"{'good_driving': 1, 'effective_defense': 1, 'c...",968.0
980.0,0.571429,0.428571,0.000000,1.0,1,1,0,2,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,{},980.0
1148.0,0.142857,0.142857,0.000000,0.285714,1,1,0,2,0.000000,0.0,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,{'good_driving': 3},1148.0
1159.0,0.250000,0.375000,0.000000,0.625,1,1,0,1,0.000000,0.5,...,0.142857,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,"{'good_driving': 1, 'clogging': 1}",1159.0


In [41]:
client.close()